# Fit emimTF2N data

EoS: PR

Parameter T dependence: Quadratic

N (total fitting parameters): 6

## Import Functions

In [1]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../../../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

# parmest (binary_param2)
from bip_fitting_functions import cuadratic

import scipy.stats as stats

## Load Data

In [2]:
data_subset = pd.read_csv('../r32_emimtf2n_subset.csv')

## Load Configuration + Load Original Parameters

In [3]:
params = pd.read_csv('Data/Fits/PR_params_quadTdep.csv',header=None)

In [4]:
params

,0
0,0.321298
1,0.842982
2,-0.553211
3,-0.610086
4,0.186268
5,-0.393545
6,0.000000
7,0.000000


In [5]:
# Take average of A, B, C (ij vs. ji) params, save
paramA = (params.iloc[0] + params.iloc[1])/2
paramB = (params.iloc[2] + params.iloc[3])/2
paramC = (params.iloc[4] + params.iloc[5])/2
paramD = 0

In [6]:
from hfc32_emimtf2n_PR import configuration 

In [7]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = GenericParameterBlock(default=configuration)
m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

The initial DOF is 7


## Apply Parameters + Plot

In [8]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [16]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(298.15)
m.fs.F101.inlet.pressure.fix(150000)
m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(0.2)

m.fs.properties.PR_kappa_A['R32','emimTf2N'] = paramA[0]
m.fs.properties.PR_kappa_A['emimTf2N','R32'] = paramA[0]
m.fs.properties.PR_kappa_B['R32','emimTf2N'] = paramB[0]
m.fs.properties.PR_kappa_B['emimTf2N','R32'] = paramB[0]
m.fs.properties.PR_kappa_C['R32','emimTf2N'] = paramC[0]
m.fs.properties.PR_kappa_C['emimTf2N','R32'] = paramC[0]

# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].unfix()

#need to use data_extended to get curve
R32_x = np.zeros((len(data_subset)))
P = np.zeros((len(data_subset)))
T = np.zeros((len(data_subset)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"] == m.fs.liq)

for i in range(len(data_subset)):
    print('i:',i)
    print('Try x_IL:',data_subset["x_emimTf2N"].iloc[i])
    m.fs.liq = data_subset["x_emimTf2N"].iloc[i]
    m.fs.F101.inlet.temperature.fix(float(data_subset["temperature"].iloc[i]))
    m.fs.F101.inlet.pressure.fix(float(data_subset["pressure"].iloc[i]))
    m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(float(data_subset["x_R32"].iloc[i])+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(float(1-(data_subset["x_R32"].iloc[i]+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(data_subset["temperature"].iloc[i]))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            R32_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R32']) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
            T[i] = data_subset["temperature"].iloc[i]
        else:
            print('Infeasible Solution')
            R32_x[i] = np.nan
            P[i] = np.nan
            T[i] = data_subset["temperature"].iloc[i]
            
    except ValueError:
        R32_x[i] = np.nan
        P[i] = np.nan
        T[i] = data_subset["temperature"].iloc[i]
        print('Solver Error')

i: 0
Try x_IL: 0.864
Feasible/Optimal Solution
i: 1
Try x_IL: 0.694
Feasible/Optimal Solution
i: 2
Try x_IL: 0.552
Feasible/Optimal Solution
i: 3
Try x_IL: 0.43
Feasible/Optimal Solution
i: 4
Try x_IL: 0.328
Feasible/Optimal Solution
i: 5
Try x_IL: 0.214
Feasible/Optimal Solution
i: 6
Try x_IL: 0.905
Feasible/Optimal Solution
i: 7
Try x_IL: 0.782
Initialization Error
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
Infeasible Solution
i: 8
Try x_IL: 0.675
Initialization Error
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
Infeasible Solution
i: 9
Try x_IL: 0.583
Initialization Error
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to

In [ ]:
fitted_data = pd.DataFrame(data=[R32_x,P/1e6,T])
fitted_data = fitted_data.T
fitted_data.columns = ['R32_x','P, MPa','T, K']
# fitted_data.to_csv('Data/Fits/PR_FittedData_quadTdep_subset_symm.csv')

In [ ]:
fitted_data

In [ ]:
error = (fitted_data['P, MPa'] - data_subset['pressure']*1E-6)

In [ ]:
error_squared = error**2

In [ ]:
SSE = np.sum(error_squared)

In [ ]:
SSE

In [ ]:
# For scaling (per original setup)
SSE*1E-3